## Compare runs, choose a model, and deploy it to a REST API

### Ref https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html

![alt](https://mlflow.org/docs/latest/assets/images/quickstart_tracking_overview-2fc1efa4bce294fc4114ce35fa34fe04.png)

* Run a hyperparameter sweep on a training script
* Compare the results of the runs in the MLflow UI
* Choose the best run and register it as a model
* Deploy the model to a REST API
* Build a container image suitable for deployment to a cloud platform

As an ML Engineer or MLOps professional, you can use MLflow to compare, share, and deploy the best models produced by the team. In this quickstart, you will use the MLflow Tracking UI to compare the results of a hyperparameter sweep, choose the best run, and register it as a model. Then, you will deploy the model to a REST API. Finally, you will create a Docker container image suitable for deployment to a cloud platform.

In [ ]:
# Setup the tracker URI
!export MLFLOW_TRACKING_URI=http://localhost:5000

In [ ]:
# install the required packages
!pip -q install hyperopt

In [ ]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

#### Now load the dataset and split it into training, validation, and test sets.

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("wine-quality-hyperopt")

In [ ]:
last_run = mlflow.last_active_run()
last_run

In [ ]:
# Retrieve the run, including dataset information
if last_run != None:
    run = mlflow.get_run(mlflow.last_active_run().info.run_id)
    dataset_info = run.inputs.dataset_inputs[0].dataset
    print(f"Dataset name: {dataset_info.name}")
    print(f"Dataset digest: {dataset_info.digest}")
    print(f"Dataset profile: {dataset_info.profile}")
    print(f"Dataset schema: {dataset_info.schema}")

In [ ]:
if last_run == None:
    print("Reading from URI.")
    source_uri = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"
    # Load dataset
    data = pd.read_csv(source_uri, sep=";",)
    mlflow_ds = mlflow.data.from_pandas(data, source=source_uri)
else:
    print("Reading the dataset used by the previous run.")
    # Load the dataset's source, which downloads the content from the source URL to the local filesystem
    dataset_source = mlflow.data.get_source(dataset_info)
    data = pd.read_csv(dataset_source.load(), sep=";")
    mlflow_ds = mlflow.data.from_pandas(data, source = dataset_source.url)

In [ ]:
data

In [ ]:

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [ ]:
train_x.shape[1]


### Then let’s define the model architecture and train the model. The train_model function uses MLflow to track the parameters, results, and model itself of each trial as a child run.

In [ ]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    model = keras.Sequential(
        [
            keras.layers.Input([train_x.shape[1]]),
            #keras.layers.Normalization(mean=np.mean(train_x), variance=np.var(train_x)),
            keras.layers.Dense(params["nodes"], activation=params["act_fn"]),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )
    
    #print(model.summary())

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


### The `objective` function takes in the hyperparameters and returns the results of the train_model function for that set of hyperparameters.

In [ ]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


Let's define the search space for `Hyperopt`. In this case, we want to try different values of `learning-rate` and `momentum`. Hyperopt begins its optimization process by selecting an initial set of hyperparameters, typically chosen at random or based on a specified domain space. This domain space defines the range and distribution of possible values for each hyperparameter. After evaluating the initial set, Hyperopt uses the results to update its probabilistic model, guiding the selection of subsequent hyperparameter sets in a more informed manner, aiming to converge towards the optimal solution.

In [ ]:
nodes = [64, 128, 32]
act_fns = ["relu", "sigmoid", "tanh"]

space = {
    "lr": hp.loguniform("lr", np.log(1e-6), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
    "nodes": hp.choice("nodes", nodes),
    "act_fn": hp.choice("act_fn", act_fns),
}


Finally, we will run the hyperparameter sweep using Hyperopt, passing in the objective function and search space. Hyperopt will try different hyperparameter combinations and return the results of the best one. We will store the best parameters, model, and evaluation metrics in MLflow.

In [ ]:
with mlflow.start_run(run_name="wine-quality-run-hyperopt"):
    # log the input artifact
    mlflow.log_input(mlflow_ds, "training")
        
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=10,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    best['nodes'] = nodes[best['nodes']]
    best['act_fn'] = act_fns[best['act_fn']]
    
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


### Serve the model locally


In [ ]:
# Install pyenv from https://github.com/pyenv/pyenv#installation
# install libffi-dev liblzma-dev libbz2-dev libssl-dev before spawning the model serving.
# if you come across _ctypes not found error,
# sudo apt-get install libffi-dev

#### MLflow allows you to easily serve models produced by any run or model version. You can serve the model you just registered by running

In [ ]:
!mlflow models serve -m "models:/wine-quality-hyperopt/2" --port 5002 --host 0.0.0.0

#### To test the model, you can send a request to the REST API using the curl command

In [ ]:
!curl -d '{"dataframe_split": {"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' -H 'Content-Type: application/json' -X POST localhost:5002/invocations

### Build a container image for your model

Most routes toward deployment will use a container to package your model, its dependencies, and relevant portions of the runtime environment. You can use MLflow to build a Docker image for your model.

#### Note: Install [nvidia-container-toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html) to use GPUs.

In [ ]:
!mlflow models build-docker --model-uri "models:/wine-quality/1" --name "qs_mlops"

This command builds a Docker image named qs_mlops that contains your model and its dependencies.

In [ ]:
!docker run -p 5002:8080 qs_mlops

This Docker run command runs the image you just built and maps port 5002 on your local machine to port 8080 in the container. You can now send requests to the model using the same curl command as before:

In [ ]:
!curl -d '{"dataframe_split": {"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' -H 'Content-Type: application/json' -X POST localhost:5002/invocations